In [1]:
import numpy as np
import pandas as pd
import nibabel as nib
import pickle
import os.path

from dipy.io import read_bvals_bvecs
from dipy.core.gradients import gradient_table
from dipy.segment.mask import median_otsu
from dipy.reconst.csdeconv import (ConstrainedSphericalDeconvModel, auto_response)
from dipy.data import get_sphere
from dipy.direction import peaks_from_model
from dipy.reconst.dti import TensorModel
from dipy.tracking.local import LocalTracking, ThresholdTissueClassifier
from dipy.tracking.streamline import Streamlines
from dipy.tracking.utils import random_seeds_from_mask

from nibabel.streamlines import save as save_trk
from nibabel.streamlines import Tractogram


root_path = "/user/pfilipia/home/inria/chu_nice_inria/patients_dmri/"


def get_patient_path(patient_id):
    return os.path.join(
        root_path,
        "patient%02d/bids/sub-patient%02d/ses-presurgical/" % (patient_id, patient_id)
    )

def get_patient_file_prefix(patient_id):
    return "sub-patient%02d_ses-presurgical_dwi" % patient_id

def get_rois_file(patient_id):
    return os.path.join(
        get_patient_path(patient_id), 'connectivity', 'rois.csv'
    )

def get_dwi_file(patient_id):
    return os.path.join(
        get_patient_path(patient_id), 'proc_local',
        get_patient_file_prefix(patient_id) + '.nii.gz'
    )

def get_bvals_file(patient_id):
    return os.path.join(
        get_patient_path(patient_id), 'proc_local',
        get_patient_file_prefix(patient_id) + '.bval'
    )

def get_bvecs_file(patient_id):
    return os.path.join(
        get_patient_path(patient_id), 'proc_local',
        get_patient_file_prefix(patient_id) + '.bvec'
    )

def load_dwi_data(patient_id):
    img = nib.load(get_dwi_file(patient_id))

    bvals, bvecs = read_bvals_bvecs(
        get_bvals_file(patient_id), get_bvecs_file(patient_id)
    )

    return img.get_data(), img.affine, gradient_table(bvals, bvecs)


def add_spherical_roi(mask, center_x, center_y, center_z, diameter):
    radius = diameter / 2
    radius_sqr = radius * radius

    for z in np.arange(-radius, radius + 1):
        for y in np.arange(-radius, radius + 1):
            for x in np.arange(-radius, radius + 1):
                mask[int(center_x + x), int(center_y + y), int(center_z + z)] = x * x + y * y + z * z <= radius_sqr


/home/patrykfi/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [1]:
patient_id = 1

dwi_data, dwi_affine, gtab = load_dwi_data(patient_id)
rois_pd = pd.read_csv(get_rois_file(patient_id))


dwi_data_masked, brain_mask = median_otsu(dwi_data)


/home/patrykfi/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# --- probabilistic tractography (from PMF) ---

csd_model = ConstrainedSphericalDeconvModel(gtab, None, sh_order=6)
csd_fit = csd_model.fit(dwi_data, mask=brain_mask)

from dipy.reconst.shm import CsaOdfModel

csa_model = CsaOdfModel(gtab, sh_order=6)
gfa = csa_model.fit(dwi_data, mask=brain_mask).gfa
tissue_classifier = ThresholdTissueClassifier(gfa, .25)

from dipy.direction import ProbabilisticDirectionGetter
from dipy.data import small_sphere

fod = csd_fit.odf(small_sphere)
pmf = fod.clip(min=0)
# prob_dg = ProbabilisticDirectionGetter.from_pmf(pmf, max_angle=30.,
#                                                 sphere=small_sphere)

diameter = 5

for stimulation_site_record in rois_pd[rois_pd['label'].str.contains('s')].iterrows():
    generated_mask = np.zeros_like(dwi_data[..., 0])
    add_spherical_roi(
        generated_mask,
        stimulation_site_record[1]['dwi_x'],
        stimulation_site_record[1]['dwi_y'],
        stimulation_site_record[1]['dwi_z'],
        diameter
    )
    
    break

    seeds = random_seeds_from_mask(generated_mask, seeds_count=2)
    # seeds = utils.seeds_from_mask(generated_mask, density=1, affine=dwi_affine)

    streamlines_generator = LocalTracking(prob_dg, tissue_classifier, seeds, affine=np.eye(4), step_size=0.5)
    streamlines = Streamlines(streamlines_generator)

    from dipy.tracking import utils

    for electrode_record in rois_pd[rois_pd['label'].str.contains('e')].iterrows():
        target_mask = np.zeros_like(dwi_data[..., 0])
        add_spherical_roi(
            target_mask,
            electrode_record[1]['dwi_x'],
            electrode_record[1]['dwi_y'],
            electrode_record[1]['dwi_z'],
            diameter
        )

        target_streamlines = utils.target(streamlines, target_mask, affine=np.eye(4))
        target_streamlines = Streamlines(target_streamlines)

        print("%s -> %s: %d" % (stimulation_site_record[1]['label'], electrode_record[1]['label'], len(target_streamlines)))
        break


/home/patrykfi/anaconda3/lib/python3.5/site-packages/dipy/reconst/csdeconv.py:162: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  r_sh = np.linalg.lstsq(self.B_dwi, self.S_r[self._where_dwi])[0]
/home/patrykfi/anaconda3/lib/python3.5/site-packages/dipy/reconst/csdeconv.py:586: UserWarning: maximum number of iterations exceeded - failed to converge
  warnings.warn(msg)


In [25]:
diameter = 5

for stimulation_site_record in rois_pd[rois_pd['label'].str.contains('s')].iterrows():
    generated_mask = np.zeros_like(dwi_data[..., 0])
    add_spherical_roi(
        generated_mask,
        stimulation_site_record[1]['dwi_x'],
        stimulation_site_record[1]['dwi_y'],
        stimulation_site_record[1]['dwi_z'],
        diameter
    )

    seeds = random_seeds_from_mask(generated_mask, seeds_count=2)
    # seeds = utils.seeds_from_mask(generated_mask, density=1, affine=dwi_affine)

    streamlines_generator = LocalTracking(prob_dg, tissue_classifier, seeds, affine=np.eye(4), step_size=0.5)
    streamlines = Streamlines(streamlines_generator)

    from dipy.tracking import utils
    from dipy.tracking.utils import length

    for electrode_record in rois_pd[rois_pd['label'].str.contains('e')].iterrows():
        target_mask = np.zeros_like(dwi_data[..., 0])
        add_spherical_roi(
            target_mask,
            electrode_record[1]['dwi_x'],
            electrode_record[1]['dwi_y'],
            electrode_record[1]['dwi_z'],
            diameter
        )

        target_streamlines = utils.target(streamlines, target_mask, affine=np.eye(4))
        target_streamlines = Streamlines(target_streamlines)
        lengths = np.array(list(length(target_streamlines)))
        
        print(
            "%s, %s, %d, %.1f, %.1f, %.1f" % (
                stimulation_site_record[1]['label'], electrode_record[1]['label'], len(target_streamlines),
                np.min(lengths), np.mean(lengths), np.max(lengths)
            )
        )
        break
    break


s00, e01, 94, 14.0, 98.8, 291.0


In [2]:
diameter = 5

generated_mask = np.zeros_like(dwi_data[..., 0])
add_spherical_roi(
    generated_mask,
    stimulation_site_record[1]['dwi_x'],
    stimulation_site_record[1]['dwi_y'],
    stimulation_site_record[1]['dwi_z'],
    diameter
)


NameError: name 'dwi_data' is not defined

In [6]:
np.sum(generated_mask)

515.0

In [23]:
import numpy.linalg as npl

def shift_point_inwards(x, y, z, affine, dist):
    world_coords = np.array(apply_affine(affine, [x, y, z]))
    world_coords *= 1 - dist / np.sqrt(np.sum(world_coords ** 2))
    shifted_coords = np.round(apply_affine(npl.inv(affine), world_coords))
    return (int)(shifted_coords[0]), (int)(shifted_coords[1]), (int)(shifted_coords[2])

In [5]:
from nibabel.affines import apply_affine
img, affine, gtab = load_dwi_data(1)

In [24]:
dwi_coords = [188, 77, 19]
shift_point_inwards(dwi_coords[0], dwi_coords[1], dwi_coords[2], affine, 5)


(183, 78, 20)

In [12]:

inv_affine = npl.inv(affine)

In [13]:
apply_affine(inv_affine, [-57.85783141,  -9.86705371, -18.68368752])

array([188.,  77.,  19.])